In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
spark.sparkContext.uiWebUrl

'http://DESKTOP-GLKIGTM.mshome.net:4040'

In [5]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [6]:
df_green.registerTempTable('green')

In [7]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [8]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7300000000002|            45|
|2020-01-01 00:00:00|  17|195.02999999999997|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25|             531.0|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|            317.27|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [9]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [10]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [11]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [12]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   3|              25.0|             1|
|2020-01-01 00:00:00|   4|1004.3000000000002|            57|
|2020-01-01 00:00:00|   7|            455.17|            38|
|2020-01-01 00:00:00|  10|             42.41|             2|
|2020-01-01 00:00:00|  12|             107.0|             6|
|2020-01-01 00:00:00|  13|1214.8000000000002|            56|
|2020-01-01 00:00:00|  14|               8.8|             1|
|2020-01-01 00:00:00|  15|             34.09|             1|
|2020-01-01 00:00:00|  17|220.20999999999998|             8|
|2020-01-01 00:00:00|  18|               5.8|             1|
|2020-01-01 00:00:00|  24|            754.95|            45|
|2020-01-01 00:00:00|  25|            324.35|            16|
|2020-01-01 00:00:00|  32|              18.0|             1|
|2020-01-01 00:00:00|  3

In [13]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [14]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [15]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [16]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [17]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  65|199.48999999999998|                  10| 409.3500000000001|                   19|
|2020-01-01 00:00:00| 161|              null|                null|           9410.21|                  488|
|2020-01-01 01:00:00|   3|46.230000000000004|                   2|              null|                 null|
|2020-01-01 02:00:00|  17|200.02999999999997|                  11|            315.57|                   16|
|2020-01-01 02:00:00| 107|              null|                null|7876.8699999999935|                  485|
|2020-01-01 02:00:00| 162|              null|                null| 3101.869999999999|                  180|
|2020-01-01 03:00:00| 234|  

In [18]:
df_join \
    .repartition(20) \
    .write.parquet('data/report/revenue/total', mode='overwrite')

In [19]:
df_join = spark.read.parquet('data/report/revenue/total')

In [20]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [21]:
df_join.show()

+-------------------+----+------------+--------------------+------------------+---------------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+------------------+---------------------+
|2021-02-21 12:00:00| 220|        null|                null| 89.19999999999999|                    3|
|2020-09-08 15:00:00| 244|      212.12|                  10|            178.44|                    6|
|2021-04-30 02:00:00|  93|        28.8|                   1|              null|                 null|
|2021-03-26 17:00:00| 180|       36.86|                   1|              null|                 null|
|2020-11-15 14:00:00|  35|        22.0|                   1|             39.76|                    1|
|2020-03-05 23:00:00|  70|        15.3|                   1|              null|                 null|
|2020-12-02 08:00:00|  36|        null|                null|             25.96|   

In [33]:
df_zones = spark.read.parquet('data/zones/')
df_zones = df_zones.withColumnRenamed('Zone','Zone_Name')

In [34]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [35]:
df_result.show()

+-------------------+----+------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|           Zone_Name|service_zone|
+-------------------+----+------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2021-02-21 12:00:00| 220|        null|                null| 89.19999999999999|                    3|       220|    Bronx|Spuyten Duyvil/Ki...|   Boro Zone|
|2020-09-08 15:00:00| 244|      212.12|                  10|            178.44|                    6|       244|Manhattan|Washington Height...|   Boro Zone|
|2021-04-30 02:00:00|  93|        28.8|                   1|              null|                 null|        93|   Queens|Flushing Meadows-...|   Boro Zone|
|2021-03-26 17:00:00| 180|       36.86|                   

In [36]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones', mode='overwrite')

In [37]:
spark.stop()